# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [103]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [104]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
def get_file_paths():
    """
    - Returns all the file path in the event_data folder 
    """
    filepath = os.getcwd() + '/event_data'

    # Create a for loop to create a list of files and collect each filepath
    for root, dirs, files in os.walk(filepath):

    # join the file path and roots with the subdirectories using glob
        return glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [105]:
def merge_csv_files():
    """
    - Reads each file in the filepath and adds all the lines to a list
    - Writes the list onto a csv file
    """
    # initiating an empty list of rows that will be generated from each file
    full_data_rows_list = [] 
    file_path_list = get_file_paths()
    # for every filepath in the file path list 
    for f in file_path_list:

        # reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)

     # extracting each data row one by one and append it        
            for line in csvreader:
                #print(line)
                full_data_rows_list.append(line) 

    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [106]:
merge_csv_files()

In [107]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [108]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace and Set Keyspace

In [109]:
def create_keyspace():
    """
    -  Create and set keyspace
    """
    try:
        session.execute(""" CREATE KEYSPACE IF NOT EXISTS nosqlproject WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}""")
    except Exception as e:
        print(e)
    
    try:
        session.set_keyspace('nosqlproject')
    except Exception as e:
        print(e)
    

In [110]:
create_keyspace()

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1

#### songs_details_by_session: Table designed to query for song metadata by session_id and item_in_session
#### Primary Key(session_id, item_in_session)
#### Since we are quering on session_id and item_in_session, we will use session_id as the partition key and the item_in_session as the clustering column.

In [111]:
create_song_details = "CREATE TABLE IF NOT EXISTS song_details_by_session (session_id int, item_in_session int, artist text, title text, length float, \
PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(create_song_details)
except Exception as e:
    print(e)

In [112]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details_by_session (session_id, item_in_session, artist, title, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"

        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))
        except Exception as e:
            print(e)


#### Do a SELECT to verify that the data have been inserted into each table

In [113]:
select_song_details = "select artist, title, length from song_details_by_session where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(select_song_details)
except Exception as e:
    print(e)
    
results = {'artist':[], 'title':[], 'length':[]}

for row in rows:
    results['artist'].append(row.artist)
    results['title'].append(row.title)
    results['length'].append(row.length)

pd.DataFrame.from_dict(results)

,artist,title,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2
#### user_history_by_session: Table designed to query for a users name and the songs they were playing given a user_id and session_id 
#### Primary(user_id, session_id, item_session)
#### Since we are quering on user_id, we'll user_id as the partition key, session_id and item_in_session as the clustering column. Using item_in_session as the clusterting column will help us sort the songs by item_in_session.

In [114]:
create_user_history_session = "CREATE TABLE IF NOT EXISTS user_history_by_session (user_id int, session_id int, \
item_in_session int, artist text, title text, first_name text, last_name text, \
PRIMARY KEY(user_id, session_id, item_in_session))"

try:
    session.execute(create_user_history_session)
except Exception as e:
    print(e)

In [115]:
# file name
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history_by_session (user_id, session_id, item_in_session, artist, title, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"

        try:
            session.execute(query, (int(line[-1]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)                

In [116]:
select_user_history = "select artist, title, first_name, last_name from user_history_by_session where user_id = 10 and session_id = 182"
try:
    rows = session.execute(select_user_history)
except Exception as e:
    print(e)

results = {'artist':[], 'title':[], 'first_name':[], 'last_name':[]}
for row in rows:
    results['artist'].append(row.artist)
    results['title'].append(row.title)
    results['first_name'].append(row.first_name)
    results['last_name'].append(row.last_name)
pd.DataFrame.from_dict(results)

,artist,title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### QUERY 3
#### user_info_by_song: A table designed to query for user details by the song title
#### Primary Key (title, user_id, iteminsession)

#### Since we query on the song title, we'll use title as the partition key. We'll user_id and item_in_session as the clustering column to create a unique key to ensure all records are inserted into the table


In [102]:
creaet_user_info_song = "CREATE TABLE IF NOT EXISTS user_info_by_song (title text, user_id int, item_in_session int, first_name text, last_name text, \
PRIMARY KEY(title, user_id, item_in_session))"

try:
    session.execute(creaet_user_info_song)
except Exception as e:
    print(e)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


In [98]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_info_by_song (title, user_id, item_in_session, first_name, last_name)"
        query = query + " VALUES( %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[8]), int(line[3]), line[1], line[4]))
        except Exception as e:
            print(e)

In [99]:
select_user_info = "select first_name, last_name from user_info_by_song where title='All Hands Against His Own'"
try:
    rows = session.execute(select_user_info)
except Exception as e:
    print(e)

results = {'first_name':[], 'last_name':[]}
for row in rows:
    results['first_name'].append(row.first_name)
    results['last_name'].append(row.last_name)
pd.DataFrame.from_dict(results)

,first_name,last_name
0,Sara,Johnson
1,Jacqueline,Lynch
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [100]:
try:
    session.execute("DROP TABLE IF EXISTS song_details_by_session")
    session.execute("DROP TABLE IF EXISTS user_history_by_session")
    session.execute("DROP TABLE IF EXISTS user_info_by_song")

except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [101]:
session.shutdown()
cluster.shutdown()